In [ ]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

In [ ]:
stocks = pd.read_csv("sp_500_stocks.csv")
stocks

In [ ]:
from secretz import IEX_CLOUD_API_TOKEN

In [ ]:
symbol = "AAPL"
api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}"
data = requests.get(api_url).json()
print(data)

In [ ]:
price = data ["latestPrice"]
print(price)
market_cap = data ["marketCap"]
print(f"{market_cap/1000000000000} Trillion")

In [ ]:
my_columns = ["Ticker", "Stock Price", "Market Capitalization", "Number Of Shares To Buy"]
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

In [ ]:
final_dataframe.append(
    pd.Series(
    [
        symbol,
        price,
        market_cap,
        "N/A"
    ],
    index = my_columns
    ),
    ignore_index = True
)

In [ ]:
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks["Ticker"][:5]:
    api_url = f"https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
        pd.Series([
            stock,
            data["latestPrice"],
            data["marketCap"],
            "N/A"
        ],
        index = my_columns),
    ignore_index = True
    )

In [ ]:
final_dataframe

In [ ]:
def chunks(lst, n):
    """Yields Successive n-sized chunks from lst"""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
    

In [ ]:
stocks = stocks[~stocks['Ticker'].isin(['DISCA', 'HFC','VIAC','WLTW'])]
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'], 
                                                   data[symbol]['quote']['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

In [ ]:
portfolio_size = input("Enter The Value Of Your Portfolio")
try:
    val = float(portfolio_size)
except ValueError:
    print("Enter an integer for the portfolio if you don't have one then Enter 0\n")
    portfolio_size = input("Enter The Value Of Your Portfolio")
    val = float(portfolio_size)

In [ ]:
position_size = val/len(final_dataframe.index)
for i in range(len(final_dataframe.index)):
    final_dataframe.loc[i , "Number of Shares to Buy"] = math.floor(position_size/final_dataframe.loc[i, "Stock Price"])
final_dataframe

#number_of_apple_shares = position_size/500
#print(math.floor(number_of_apple_shares))

In [ ]:
writer = pd.ExcelWriter("recommended trades.xlsx", engine = "xlsxwriter")
final_dataframe.to_excel(writer, "Recommended Trades", index = False)

In [ ]:
background_color = "#0a0a23"
font_color = "#ffffff"

string_format = writer.book.add_format(
    {
        "font_color" : font_color,
        "bg_color" : background_color,
        "border" : 1
    }
)
dollar_format = writer.book.add_format(
    {
        "num_format" : "$0.00",
        "font_color" : font_color,
        "bg_color" : background_color,
        "border" : 1
    }
)
integer_format = writer.book.add_format(
    {
        "num_format" : "0",
        "font_color" : font_color,
        "bg_color" : background_color,
        "border" : 1
    }
)


In [ ]:
column_formats = {
    "A" : ["Ticker", string_format],
    "B" : ["Stock Price", dollar_format],
    "C" : ["Market Capitalisation", dollar_format],
    "D" : ["Number of Shares to Buy", integer_format]
}
for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Tradess'].write(f'{column}1', column_formats[column][0], string_format)

In [ ]:
writer.save()